In [1]:
def create_features(*keys):
    features = {key: 0 for key in keys}
    return features

def set_list01():
    return create_features("NF", "NC", "NR", "NP")

def set_list02():
    return create_features("NPt_s", "NAdd_s", "NPt_ds", "NAdd_ds")

def set_list03():
    return create_features("NStr", "NStrM", "NStrT", "NStrI", "NStrC")

def set_list04():
    return create_features("NFRec", "NCRec", "NIFPar", "NRRec", "NRNRec")

def set_list05():
    return create_features("NSizeof", "NMalloc", "NFree")


In [2]:
def compileRegex(code, regex, printador=False):
    p = re.compile(regex)

    ls = []
    for line in code:
        for correspondencia in p.finditer(line):
            ls.append(correspondencia.group())
            if printador: 
                print(correspondencia.group())
    return(ls)

In [3]:
# LISTA 01
# -------------------------
def RE_NF(ls_types):
    types = '|'.join(ls_types)
    # não contabiliza a função main 
    # trata espaço entre main e parâmetros
    # pode terminar com { ou não, quando abre chaves está na linha seguinte}
    return fr'^(?:{types})\s.+(?<!main)(?<!main )\(.*\)\s*{{?$'

def RE_NR():
    return(r'^return(\s|\().+;')

# LISTA 02
# -------------------------
# devolve uma lista com todas as assinaturas
# ex.: ['void tiraDois (int* num, short int* r1){', 'void tiraDois (int* num, short int* r1){']
def RE_NPt_s(ls_types):
    types = '|'.join(ls_types)
    return(r'^(?:{types})\s.+(?<!main)(?<!main )\(.+\*.+\)\s*{?$')

# LISTA 05
# -------------------------
def RE_sizeof():
    return(r'^.+sizeof\s*\(.+\);$')

def RE_malloc():
    return(r'^.+malloc\s*\(.+\);$')

def RE_free():
    return(r'^.*free\s*\(.+\);$')


In [4]:
##-------------------------------------------------------------------------------
def get_list01(features01, code, ls_calls, ls_types):
    #--------------------------
    features01["NF"] += len(compileRegex(code, RE_NF(ls_types)))
    features01["NC"] += len(ls_calls)
    features01["NR"] += len(compileRegex(code, RE_NR()))
    features01["NP"] += get_NOP(compileRegex(code, RE_NF(ls_types)), ls_types)
    #--------------------------  
    return(features01)

##-------------------------------------------------------------------------------
def get_list02(features02, code, ls_functions, ls_calls, ls_func_call, ls_types):  
    #--------------------------
    features02['NPt_s'] += sum([x.count('*') for x in compileRegex(code, RE_NPt_s(ls_types))])
    features02['NAdd_s'] += get_NAdd_s(ls_calls, ls_types)
    #[print(len(x.split(','))) for x in compileRegex(code, RE_NPt_ds(), True)]
    features02['NPt_ds'] += get_NPt_ds(code)
    features02['NAdd_ds'] += get_NAdd_ds(code, ls_calls, ls_func_call, ls_types)
    #--------------------------         
    return(features02)
            
##-------------------------------------------------------------------------------
def get_list03(features03, code):      
    #-------------------------- 
    ls_structs = get_struct(code)
    ls_alias, ls_lines = get_typedef(code)
    ls_instances_s, ls_instances_ds = get_instance(code, ls_alias, ls_lines)

    features03['NStr'] += len(ls_structs)
    features03['NStrM'] += get_NOSrtM(ls_structs)
    features03['NStrT'] += len(ls_alias)
    features03['NStrI'] += len(ls_instances_s)
    features03['NStrC'] += get_NOStrC(code, ls_instances_s)
    #-------------------------- 
    return(features03)

##-------------------------------------------------------------------------------
def get_list04(features04, code, ls_functions):    
    features04['NFRec'] += get_NOFRec(ls_functions)
    features04['NCRec'] += get_NOCRec(code, ls_functions)
    features04['NIFPar'] += get_NOIFPar(ls_functions)
    features04['NRRec'] += get_NORRec(ls_functions)    
    features04['NRNRec'] += get_NORNRec(ls_functions)
    #--------------------------     
    return(features04)
    
##-------------------------------------------------------------------------------
def get_list05(features05, code):      
    #--------------------------
    features05['NSizeof'] += len(compileRegex(code, RE_sizeof()))
    features05['NMalloc'] += len(compileRegex(code, RE_malloc()))
    features05['NFree'] += len(compileRegex(code, RE_free()))
    #-------------------------- 
    return(features05)

### Features from the L1

In [ ]:
## Guarda o número da linha onde inicia uma função
## Utilizada em get_functions
##------------------------------------------------------------------------------- 
def get_flags(code, ls_types):
    ls_signatures = compileRegex(code, RE_NF(ls_types))
    ls_flags = []
    #--------------------------
    for sig in ls_signatures:   
        for linha, frase in enumerate(code):
            if sig in frase:
                ls_flags.append(linha)
                break
    #--------------------------
    return ls_flags


## Guarda o corpo das funções
## Utilizada para extrair FLC
##------------------------------------------------------------------------------- 
def get_functions(codigo, ls_types):
    ls_functions = []
    ls_flags = get_flags(codigo, ls_types)
    #--------------------------
    for i, flag in enumerate(ls_flags):
        ls_function = []
        #--------------------------
        if i+1 < len(ls_flags):
            fim = ls_flags[i+1]
        else:
            fim = len(codigo)
        #--------------------------
        for frase in codigo[flag:fim]:
            if "main(" in frase.replace(" ", ""):
                break
            else:    
                ls_function.append(frase)
        #--------------------------
        if ls_function:
            ls_functions.append(ls_function)
    #--------------------------
    return ls_functions


## Faz a contagem de linhas por função
## Utilizada diretamente na extração de FLC
##------------------------------------------------------------------------------- 
def get_sumfloc(ls_functions):
    count = 0
    #--------------------------
    for funcao in ls_functions:
        count += len(funcao)
    #--------------------------
    return(count)


## Guarda o número de parâmetros
## Utilizada diretamente na extração de NP
##------------------------------------------------------------------------------- 
def get_NOP(code, ls_types):
    ls_signatures = compileRegex(code, RE_NF(ls_types))
    count = 0
    #--------------------------
    for frase in ls_signatures:
        # espaço reservado aos parâmetros não pode estar vazio
        if not frase[frase.find('(')+1:frase.find(')')-1] == "":                
            count += len(frase.split(","))
    #--------------------------
    return(count)


# ATENCAO: não contabiliza a chamada da biblioteca
##------------------------------------------------------------------------------- 
def get_calls(code, ls_types):
    ls_signatures = compileRegex(code, RE_NF(ls_types))
    ls_calls = []
    #--------------------------    
    for sig in ls_signatures:
        for linha in code:

            # encontra nome da função
            nome_da_funcao = sig[sig.find(" ")+1:sig.find("(")+1]
            if nome_da_funcao in linha:

                # descarta protótipos
                prototipo = "{};".format(sig.replace("{",""))
                if not linha == prototipo:

                    # descarta assinaturas
                    if not linha == sig:
                        ls_calls.append(linha)
    #--------------------------                
    return(ls_calls) 


## --------------------------------------------------------------------------------------------------
def get_CC(id_stud, folder_entrada):
    #--------------------------
    dir_stud = "{:02d}".format(id_stud)    
    caminho = os.path.join(folder_entrada, dir_stud)
    os.system('lizard {} > z-lizard.log'.format(caminho))
    cc = load_file('z-lizard.log')
    cc = cc[-1].split()[2]
    #--------------------------    
    return(cc)

In [12]:
## --------------------------------------------------------------------------------------------------
## INPUT: recebe todas as tarefas de um aluno por lista de exercícios (formato lista de lista)
## BODY: especifica cada código do aluno, 
## BODY: procura os tipos de dados declarados na frase
## BODY: se encontrar, procura o tipo de dado nas primeiras letras da frase, e procura por parenteses
## BODY: retira as chaves, stopwords
## OUTPUT: (ls_assinaturas) retorna as assinaturas das funções encontradas (formato lista de lista)
## OUTPUT: ex: [['int tiraDois(int *num)', 'int main()'], ['void somaQuatro(int *num)'...]]
## OUTPUT: (count) retorna a contagem total de funções
## OUTPUT: (ls_corpos) retorna o corpo das funções encontradas no código
## --------------------------------------------------------------------------------------------------
def get_signatures(codigo, ls_types):
    ls_assinaturas = []
    ls_flags = []
    #--------------------------
    for id_line, frase in enumerate(codigo):
        for t in ls_types:
            if t in frase[0:len(t)] and "(" in frase and ")" in frase:
                # artificio para não contabilizar os protótipos e as chamadas 
                try:
                    # procura chaves na mesma linha ou na linha seguinte
                    if "{" in frase or "{" in codigo[id_line+1]:
                        if "main" in frase:
                            ls_flags.append(id_line)                            
                        else:
							# encontrou uma função
                            ls_assinaturas.append(frase)
                            #print(frase)
                            
                            # guarda a linha (para buscar o corpo)
                            ls_flags.append(id_line)
                            
                        # quebra laço de tipos e passa para a próxima linha
                        break
                except IndexError as error:
					## Atenção: aluno 36 tem um bib apenas com um protótipo (por isso essa except)
                    print(">>>>>> tipo: {}, frase: {} <<<<<<".format(t, frase))
    #--------------------------
    return(ls_assinaturas, ls_flags)

In [13]:
## estou vendo se o codigo (especifico) usa alguma biblioteca propria
## se isso acontecer, preciso tratar isso nas chamadas??
## aqui eu nao possuo o codigo da biblioteca
## ele vai retornar a lista dos nomes das bibliotecas encontradas??

##------------------------------------------------------------------------------- 
def get_command(codigo, cmd):
    ls_cmd = []
    #--------------------------
    for frase in codigo:
        if cmd in frase:
            ls_cmd.append(frase)
    #--------------------------            
    return(ls_cmd)

## usar assinaturas do código 3 no código veiculo (ex aluno 1, turma 3, lista 1)
def get_calls_libs(codigo, ls_name_files):
    ls_libs_code = get_command(codigo, '#include')
    #ls_libs = []
    ls_libs = ""
    #--------------------------
    for lib_code in ls_libs_code:
        if not "<" in lib_code: 
            for name_code in ls_name_files:
                if name_code in lib_code:                    
                    #ls_libs.append(name_code)
                    ls_libs = name_code
    #--------------------------                    
    return ls_libs

### Features from the L2

In [1]:
## --------------------------------------------------------------------------------------------------
## INPUT: recebe todas as tarefas de um aluno por lista de exercícios (formato lista de lista)
## BODY: especifica cada código do aluno, frase e palavras (quebrando por parametro)
## BODY: procura por ponteiros diretos (*) e vetores/matrizes ([])
## OUTPUT: retorna a contagem total de ponteiros presentes nas assinaturas
## --------------------------------------------------------------------------------------------------
def get_NPt_ds(code, ls_types):
    ls_signatures = compileRegex(code, RE_NF(ls_types))    
    count = 0
    #-------------------------- 
    for frase in ls_signatures:
        for palavra in frase.split(","):
            if "[" in palavra and "]" in palavra:
                count += 1
    #-------------------------- 
    return count

## --------------------------------------------------------------------------------------------------
def get_NAdd_s(ls_calls):
    count = 0
    #-------------------------- 
    for call in ls_calls:
        count += call.count("&")            
    #-------------------------- 
    return count

## --------------------------------------------------------------------------------------------------
def get_var_lc(codigo, ls_types):
    count = 0
    lista = []
    
    for frase in codigo:
        for t in ls_types:
            if t in frase[0:len(t)] and not "(" in frase:
                for variavel in frase.split(","):
                    lista.append(variavel.replace(";", "").replace("{} ".format(t), ""))

                count+=len(lista)                
                # se encontrar um tipo, pode sair do laço, não tem dois tipos na mesma linha
                break 
                
    return (lista, count)

In [2]:
def get_func_call(code, ls_flags, ls_types):
    ls_signatures = compileRegex(code, RE_NF(ls_types))    
    func_call = []
    function = []
    for call in get_calls(code, ls_types):                        
        #print("call: {}".format(call))
        for i, flag in enumerate(ls_flags):
            ls_function = []
            #--------------------------
            if i+1 < len(ls_flags):
                fim = ls_flags[i+1]
            else:
                fim = len(code)

            for frase in code[flag+1:fim]:
                for sig in ls_signatures:
                    declaracao = sig[sig.find(" ")+1:sig.find("(")+1]
                    if declaracao in frase:
                        
                        #print("dec_sig: {}".format(declaracao))
                        for linha in code[flag:fim]: 
                            function.append(linha)                    
                        func_call.append(function)
                        break
                    if not function == "":
                        function = []
                        break
    #--------------------------    
    return func_call

## --------------------------------------------------------------------------------------------------
def get_NAdd_ds(code, ls_calls, ls_func_call, ls_types):
    ls_signatures = compileRegex(code, RE_NF(ls_types))    
    count = 0
    #--------------------------    
    for call, funcao in zip(ls_calls, ls_func_call):
        for assinatura in ls_signatures:
            declaracao_ass = assinatura[assinatura.find(" ")+1:assinatura.find("(")]
            if declaracao_ass in call:            
                dec_call = call[call.find(declaracao_ass):]
                args = dec_call[dec_call.find("(")+1:call.find(")")].replace(" ", "").replace(")", "").replace(";", "")
                for argumento in args.split(","):
                    variaveis, lixo = get_var_lc(funcao, ls_types)
                    for var in variaveis:    
                        if "{}[".format(argumento) in var:
                            #print(assinatura)
                            count+=1
    #--------------------------
    return count

### Features from the L3

In [ ]:
def get_struct(code):
    ls_structs = []
    ls_str = []
    flag = 1
    #--------------------------
    for id_line, line in enumerate(code):
        if flag:
            if line.strip().startswith("struct"):
                if line.strip().endswith("{") or code[id_line+1].strip().__eq__("{"):
                    ls_str.append(line)
                    flag = 0
        elif line.strip().startswith("}"):
                ls_str.append(line)
                ls_structs.append(ls_str)
                ls_str = []
                flag = 1
        else:
            ls_str.append(line)
    #--------------------------
    return(ls_structs)

## --------------------------------------------------------------------------------------------------
def get_NOSrtM(ls_structs):
    count = 0
    #--------------------------
    for struct in ls_structs:
        for line in struct:
            if not line.strip().startswith("struct"):
                if not line.strip().endswith("{") and not line.strip().startswith("}"):
                    for member in line.split(","):
                        if member: # member do not be empty
                            count += 1
    #--------------------------
    return(count)

## --------------------------------------------------------------------------------------------------
def get_typedef(code):
    position = 3
    ls_alias = []
    ls_lines = []
    #--------------------------    
    for id_line, line in enumerate(code):
        if line.strip().startswith("typedef struct"):
            if line.strip().endswith(";"):
                ls_alias.append(line.split()[position].replace(";",""))
                ls_lines.append(id_line)                
    #--------------------------
    return(ls_alias, ls_lines)

## --------------------------------------------------------------------------------------------------
def get_instance(code, ls_alias, ls_lines):
    position = 3
    ls_instances_s = []
    ls_instances_ds = []    
    #--------------------------    
    for id_line_alias, alias in zip(ls_lines, ls_alias):
        for id_line, line in enumerate(code[id_line_alias+1:]):
            if line.strip().startswith(alias):
                if line.strip().endswith(";"):
                    for instance in line.strip().replace(";","").replace(alias,"").split(","):
                        if "[" in instance:
                            ls_instances_ds.append(instance[:instance.find("[")+1].strip())
                        else:
                            ls_instances_s.append("{}.".format(instance.strip()))
    #--------------------------                     
    return(ls_instances_s, ls_instances_ds) 

## --------------------------------------------------------------------------------------------------
def get_NOStrC(code, ls_instances):
    count = 0
    #--------------------------     
    for instance in ls_instances:
        for line in code:
            if instance in line:
                count += line.count(instance)
    #--------------------------                 
    return count

### Features from the L4

In [1]:
def get_RecType(ls_functions):
    countH = 0
    countT = 0
    #--------------------------
    for function in ls_functions:
        if is_recursiveFunction(function):
            # verificar se tem algo entre o Return e a função recursiva, algo associado a operador aritmético (head)
            # se não tem return, tem alguma instrucao após a chamada recursiva? (head)
            # nenhuma das anteriores (cauda)
            
            #? parece que precisa delimitar o bloco do if, na qual a chamada recursiva está presente
            pass
    

##------------------------------------------------------------------------------- 
def get_NOFRec(ls_functions):
    count = 0
    #--------------------------    
    for function in ls_functions:
        if is_recursiveFunction(function):
            count+=1
        #print(count)
    #-------------------------- 
    return(count)

##------------------------------------------------------------------------------- 
def get_NOCRec(code, ls_functions):
    count = 0
    #--------------------------    
    for function in ls_functions:
        if is_recursiveFunction(function):
            sig = get_signatureFunction(function)
            for line in code:
                if not line == function[0]:
                    if sig in line:
                        count += 1            
    #--------------------------
    return(count)

##------------------------------------------------------------------------------- 
def get_NOR_diff(ls_functions):
    count = 0
    ls_returns = []
    ls_uniq = []
    #--------------------------    
    for function in ls_functions:
        if is_recursiveFunction(function):
            for line in function:
                if line.strip().startswith("return"):
                    ls_returns.append(line)
            ls_uniq.extend(list(set(ls_returns)))
    #--------------------------              
    return(len(ls_uniq))

# OUTPUT: just the name function, plus parenthesis (ex: 'recursivo(')
##------------------------------------------------------------------------------- 
def get_signatureFunction(function):
    #--------------------------    
    sig = function[0]
    sig = sig[sig.find(" ")+1:sig.find("(")+1]
    #--------------------------    
    return(sig)

# OUTPUT: return zero when is not a recursive function, or the recursive function count
##------------------------------------------------------------------------------- 
def is_recursiveFunction(function):
    count = 0
    #--------------------------     
    sig = get_signatureFunction(function)
    #--------------------------         
    for line in function[1:]:
        if sig in line:
            count+=1
    #--------------------------
    return(count)

##------------------------------------------------------------------------------- 
def get_parameters(sig):
    ls_parameters = []
    #--------------------------
    # RULE: space reserved to parameters is not empty
    parameters = sig[sig.find('(')+1:sig.find(')')]
    if parameters:
        for par in parameters.split(","):
            par = par.strip()
            ls_parameters.append(par[par.find(" ")+1:])
    #--------------------------
    return(ls_parameters)


##-------------------------------------------------------------------------------
def get_NOIFPar(ls_functions):
    count = 0
    #--------------------------    
    for function in ls_functions:
        if is_recursiveFunction(function):
            ls_parameters = get_parameters(function[0])
            # RULE: the first function line is the signature, thus it is disregarded.            
            for line in function[1:]:
                if "if(" in line.replace(" ", ""):
                    for parameter in ls_parameters:
                        if parameter in line:
                            #print("parameter: {}".format(line))
                            count+=1
                            break
    #--------------------------                        
    #print("NOPrintPar: {}".format(count))
    return(count)


##-------------------------------------------------------------------------------
def get_NORRec(ls_functions):
    count = 0
    #--------------------------    
    for function in ls_functions:
        if is_recursiveFunction(function):            
            funtionName = get_signatureFunction(function)
            # RULE: the first function line is the signature, thus it is disregarded.            
            for line in function[1:]:
                if line.strip().startswith("return"):
                    if funtionName in line:
                        count+=1
    #--------------------------
    return(count)


##-------------------------------------------------------------------------------
def get_NORNRec(ls_functions):
    count = 0
    #--------------------------    
    for function in ls_functions:
        if is_recursiveFunction(function):            
            funtionName = get_signatureFunction(function)
            # RULE: the first function line is the signature, thus it is disregarded.            
            for line in function[1:]:
                if line.strip().startswith("return"):
                    if not funtionName in line:
                        count+=1
                        break
    #--------------------------
    return(count)


##-------------------------------------------------------------------------------
def get_NOPrintPar(ls_functions):
    count = 0
    #--------------------------    
    for function in ls_functions:
        if is_recursiveFunction(function):
            ls_parameters = get_parameters(function[0])
            # RULE: the first function line is the signature, thus it is disregarded.
            for line in function[1:]:
                if "printf" in line: 
                    for parameter in ls_parameters:
                        if parameter in line:
                            count+=1
    #--------------------------                        
    #print("NOPrintPar/function: {}".format(count))    
    return(count)          